In [ ]:
import numpy as np
import pandas as pd
from earthscopestraintools.mseed_tools import ts_from_mseed
from earthscopestraintools.gtsm_metadata import GtsmMetadata
from earthscopestraintools.timeseries import Timeseries, plot_timeseries_comparison

import matplotlib.pyplot as plt

import logging
logger = logging.getLogger()
logging.basicConfig(
        format="%(message)s", level=logging.INFO
    )

%load_ext autoreload
%autoreload 2

**Instructions for the activity:** 

For these partner exercises, you will have a station and date range to explore. Your goal is to identify an interesting signal, using ```earthscopestraintools``` to better visualize and understand potential causes of the anomolous strain. You may explore other datasets and conduct simple research to complement your work (e.g., are there other strainmeters nearby to analyze? where is the instrument and what could be causing the signal?) - however, we request that you do NOT dive into the scientific literature, and remain data driven in your explorations. We suggest you start with a 1 Hz sampling rate for <1 mo of data, and 10 minute sampling period for >1 mo of data. You may decide on a better sampling rate for your signal after preliminary analyses. **Compile information for your projects with visuals in a mini powerpoint presentation to share with the group - please share your processing workflow along with your preliminary conclusions, as well as any code, instrumental, or other complications you encountered and how you overcame the issues.**

Other resources you may find useful for comparison: 
- [UNR map browse of GNSS stations with displacement timeseries](http://geodesy.unr.edu/NGLStationPages/gpsnetmap/GPSNetMap.html)
- [EarthScope map browse of stations](https://www.unavco.org/instrumentation/networks/map/map.html#!/)
- [USGS earthquake catalogue](https://earthquake.usgs.gov/earthquakes/search/)

### Seiches at B944

>Luttrell, K., Mencin, D., Francis, O. and Hurwitz, S. (2013), Constraints on the upper crustal magma reservoir beneath Yellowstone Caldera inferred from lake-seiche induced strain observations, Geophys. Res. Lett., 40, 501–506, doi:10.1002/grl.50155

Suggestions: 
- Acquire metadata and data.
- Remove trend, tides, and pressure from the signal. 
- Zoom in on the signal and characterize its period, amplitude, and emergence.
- Take a look at the signal transformed to areal, differential, and shear strains. 
- Where is the station? What is around it that could be causeing the anomolous strain?
- Locate the same signal in any neighboring stations, if there are any (suggest looking on the earthscope site for station locations).

Starting point: 
```
network = 'PB'
station = 'B944'
start = '2009-07-01'
end = '2009-07-20'
```



In [ ]:
# Gather data for seiches

network = 'PB'
station = 'B944'
start = '2009-07-01'
end = '2009-07-20'

meta = GtsmMetadata(station=station,network=network)

strain_raw = ts_from_mseed(network=network, station=station, location='T0', channel='LS*', start=start, end=end)
gauge_microstrain = strain_raw.linearize(reference_strains=meta.reference_strains, gap=meta.gap)

gauge_microstrain.stats()
gauge_microstrain.plot()

## 2019 Ridgecrest Earthquake Sequence

> Hanagan et al., In Review.

Two large magnitude earthquakes from the 2019 Ridgecrest Sequence with aftershocks and significant postseismic strain accumulation. 

Suggestions: 
- Acquire metadata and data
- Remove trend, tides, and pressure from the signal. 
- Look at the signal from the gauges as transformed areal, differential, and shear strains.
- Zoom in on the signal(s) and characterize what is happening before, during, and after the signal. 
- Where is the station? What is around it? What other data can help you tell what is happening at that time?
- Locate the same signal in any neighboring stations that might corroborate findings. 

Starting point: 
```
network = 'PB'
station = 'B921'
start = '2019-07-01'
end = '2019-07-20'
```

In [ ]:
network = 'PB'
station = 'B921'
start = '2019-07-01'
end = '2019-07-20'

meta = GtsmMetadata(station=station,network=network)

strain_raw = ts_from_mseed(network=network, station=station, location='T0', channel='LS*', start=start, end=end)
gauge_microstrain = strain_raw.linearize(reference_strains=meta.reference_strains, gap=meta.gap)

gauge_microstrain.stats()
gauge_microstrain.plot()

## Slow slip in Cascadia

> Hawthorne, J. C., and A. M. Rubin (2010), Tidal modulation of slow slip in Cascadia, J. Geophys. Res., 115, B09406, doi:10.1029/2010JB007502.

B004 will also show a slow slip event. 

Suggestions: 
- Acquire metadata and data (note the long period of time - 10 minute sample period might be a good starting point).
- Remove trend, tides, and pressure from the signal. 
- Look at the signal from the gauges as transformed areal, differential, and shear strains.
- Zoom in on the signal(s) and characterize what is happening before, during, and after the signal. 
- Where is the station? What is around it? What other data can help you tell what is happening at that time?
- Locate the same signal in any neighboring stations that might corroborate findings. 

Starting point: 
```
network = 'PB'
station = 'B007'
start = '2008-04-01'
end = '2008-07-01'
```

In [ ]:
network = 'PB'
station = 'B007'
start = '2008-04-01'
end = '2008-07-01'

meta = GtsmMetadata(station=station,network=network)

strain_raw = ts_from_mseed(network=network, station=station, location='T0', channel='RS*', start=start, end=end)
gauge_microstrain = strain_raw.linearize(reference_strains=meta.reference_strains, gap=meta.gap)

gauge_microstrain.stats()
gauge_microstrain.plot()

In [ ]:
# Corrections
tide_c = gauge_microstrain.calculate_tide_correction(tidal_parameters=meta.tidal_params,longitude=meta.longitude)

atmp_raw = ts_from_mseed(network=network,station=station,location='TS',channel='RDO',
                     scale_factor=0.001,start=start,end=end)
atmp = atmp_raw.interpolate(new_index=gauge_microstrain.data.index)
atmp_c = atmp.calculate_pressure_correction(response_coefficients=meta.atmp_response)
trend_c = gauge_microstrain.linear_trend_correction(method='median')

In [ ]:
# Corrected regional strains
corrected_microstrain = gauge_microstrain.apply_corrections([tide_c,atmp_c,trend_c])
reg_strain = corrected_microstrain.apply_calibration_matrix(calibration_matrix=meta.strain_matrices['lab'])

In [ ]:
reg_strain.plot()

## Rain

Canitano Floods September 15th and 16th, 2023.

Suggestions: 
- Acquire metadata and data.
- Remove trend, tides, and pressure from the signal. 
- Look at the signal from the gauges as transformed areal, differential, and shear strains.
- Zoom in on the signal(s) and characterize what is happening before, during, and after the signal. 
- Where is the station? What is around it? What other data can help you tell what is happening at that time?
- Locate other data that may be contributing to the signal, including data from neighboring stations that might corroborate findings. 

Starting point: 
```
network = 'PB'
station = 'B007'
start = '2008-04-01'
end = '2008-07-01'
```



In [ ]:
network = 'IV'
station = 'TSM2'
start = '2022-09-09'
end = '2022-09-17'

meta = GtsmMetadata(station=station,network=network)

strain_raw = ts_from_mseed(network=network, station=station, location='T0', channel='LS*', start=start, end=end)
gauge_microstrain = strain_raw.linearize(reference_strains=meta.reference_strains, gap=meta.gap)

gauge_microstrain.stats()
gauge_microstrain.plot()

## Turkey Earthquake

Teleseismic strains from the Turkey earthquake sequence, February 6th, 2023. 

Suggestions: 
- Acquire metadata and data.
- Remove trend, tides, and pressure from the signal. 
- Look at the signal from the gauges as transformed areal, differential, and shear strains.
- Zoom in on the signal(s) and characterize what is happening before, during, and after the signal. 
- Where is the station? What is around it? What other data can help you tell what is happening at that time?
- Gather other data that describes what may be contributing to the signal, including data from neighboring stations that might corroborate findings. 

Starting point: 
```
network = 'IV'
station = 'TSM6'
start = '2023-02-01'
end = '2023-02-10'
```


In [ ]:
network = 'IV'
station = 'TSM6'
start = '2023-02-01'
end = '2023-02-10'

meta = GtsmMetadata(station=station,network=network)

strain_raw = ts_from_mseed(network=network, station=station, location='T0', channel='LS*', start=start, end=end)
gauge_microstrain = strain_raw.linearize(reference_strains=meta.reference_strains, gap=meta.gap)

gauge_microstrain.stats()
gauge_microstrain.plot()

## Additional Activity

Find another (or several!) creep events on B073, or another station you expect to find creep on. 